In [1]:
import numpy as np

In [9]:
##wb是模型本身的参数因此放在构造函数中无需手动进行更改 其他则由输入的x决定
class Affine:
    def __init__(self,w,b):
        self.w=w
        self.b=b
        self.x=None
        self.dw=None
        self.db=None
        self.dx=None
    def forward(self,x):
        if x.dim==1:
            self.x = x.reshape(1,-1)
        output=np.dot(self.x, self.w)+self.b
        return output
    def backward(self,dout):
        self.dx=np.dot(dout,self.w.T)
        self.dw=np.dot(self.x.T,dout)
        self.db=np.sum(dout,axis=0)

class Relu:
    def __init__(self):
        self.mask=None
    def forward(self,x):
        self.mask=x>0
        out=self.mask*self.x
        return out
    def backward(self,dout):
        dx=self.mask*dout
        return dx
    
class Softmaxwithloss:
    def __init__(self,t):
        self.t=t
    def forward(self,x):
        out=np.exp(x-7)/np.sum(np.exp(x-7))
        return out
    def loss(self,x):
        out=self.forward(x)
        return -self.t*np.log(out+1e-7)/x.size
        

In [ ]:
class MultiLayersNetwork:
    def __init__(self, input_size, output_size, hidden_size_list=None):
        if hidden_size_list is None:
            self.hidden_size_list = [100, 100, 100]
        self.input_size = input_size
        self.hidden_size_list = hidden_size_list
        self.output_size = output_size
        self.params=dict()
        self.sourcedata=None
        self.layers= dict()
        #生成层
        parameter_size_list=self.hidden_size_list.insert(0,self.input_size)
        parameter_size_list.append(self.output_size)
        
        ##倒数第二层之前全部用relu
        for i in range(len(parameter_size_list)-2):
            self.params['W'+str(i)]=np.random.rand(parameter_size_list[i],parameter_size_list[i+1])
            self.params['b'+str(i)]=np.random.rand(parameter_size_list[i],parameter_size_list[i+1])
            #每一层就自己层的权重和偏置
            self.layers['affine'+str(i)]=Affine(self.params['W'+str(i)],self.params['b'+str(i)])
            self.layers['Activation_function']=
        

    def forward(self, x):
        for j in range(len(self.params)//2):
            layer_name='layer'+str(j)
            self.layers[layer_name]=Affine(x=x,w=self.,b=)
            